In [7]:
import cv2
import numpy as np

from boxmot import OcSort

# Initialize the tracker
tracker = OcSort(
    asso_func="centroid",
    min_hits=10,
    asso_threshold=0.98,
    det_thresh = 0.7,
    max_age=20,
    use_byte=True,
    Q_xy_scaling = 0.01,
    Q_s_scaling = 0.0001,
)

def get_parabolic_array(i):
    """
    Generates coordinates where x increases linearly,
    y follows a parabolic curve (y = base + coeff * i^2),
    and the angle matches the tangent of the curve.
    """
    # -- FIRST ROW --
    # Define x1, y1
    x1 = 144 + i
    y1 = 212 + 0.01 * (i ** 2)
    
    # Slope = 2 * coeff * i => for 0.01 * i^2, slope = 2*0.01*i = 0.02*i
    slope1 = 0.02 * i
    # Angle in radians
    angle1 = np.arctan(slope1)
    
    # -- SECOND ROW --
    # Define x2, y2 with a different coefficient
    x2 = 425 + i
    y2 = 281 + 0.02 * (i ** 2)
    
    # Slope for 0.02 * i^2 is 2*0.02*i = 0.04*i
    slope2 = 0.04 * i
    # Angle in radians
    angle2 = np.arctan(slope2)
    
    # Build the array
    det = np.array([
        [x1, y1, 45, 30, angle1, 0.82,  0],   # row 1
        [x2, y2, 45, 30, angle2, 0.72, 65]    # row 2
    ])
    
    return det


for i in range(0, 100):

    #frame = cv2.imread(str(img_dir / (file.stem + '.png')))
    frame = np.zeros((1080,1080,3))
    
    det = get_parabolic_array(i)

    # Update the tracker
    res = tracker.update(det, frame)  # --> M X (x, y, x, y, id, conf, cls, ind)
    
    # Plot tracking results on the image
    tracker.plot_results(frame, show_trajectories=True, fontscale=2, thickness=4)

    # Display the frame
    cv2.imshow('BoXMOT', frame)

    print(len(tracker.active_tracks),end='\r')

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key ==27:
        break

cv2.destroyAllWindows()